In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import SpectralClustering

In [ ]:
df_raw = pd.DataFrame(gpd.read_file('../vectors/buffer_trees_moved.geojson').drop(['geometry'], axis=1)).set_index('id')
# df_raw = df_raw.loc[df_raw['rep']==1]
df_raw = df_raw.groupby('name', as_index=False).mean()
df_raw

In [ ]:
df_raw['trat'] = [x[:2] for x in df_raw['name'].values]
df_raw

In [ ]:
result = df_raw.loc[:,['name']]
df = df_raw.copy()
y = df.loc[:,['trat']]
df

In [ ]:
df_filtered = df.iloc[:,2:-1]
# df_filtered = df_filtered.loc[:,['_min', '_minority', '_sum', '_mean', '_median', '_max']]
X = (df_filtered-df_filtered.min())/(df_filtered.max()-df_filtered.min())
X
# df_filtered

In [ ]:
array = np.array(X)

kmeans = KMeans(
    n_clusters=8,
    n_init=10,
    max_iter=500,
)
predict = kmeans.fit_predict(X)

df_kmeans = result.copy()
df_kmeans['kmeans'] = predict

In [ ]:
model = SpectralClustering(n_clusters=8, affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
df_kmeans['spectral'] = labels

In [ ]:
df_kmeans.sort_values(by='name').to_csv('../vectors/buffer_trees_moved_results.csv')

In [ ]:
gdf = gpd.read_file('../vectors/buffer_trees_moved.geojson')
# gdf = gdf.groupby('name', as_index=False).mean()
# gdf = gdf.loc[gdf['rep']==1]
gdf['kmeans'] = predict
gdf['spectral'] = labels
gdf.to_file('../vectors/buffer_trees_class.geojson', driver='GeoJSON')

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=6)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(6,'Score'))  #print 10 best features

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
'_min', '_minority', '_sum', '_mean', '_median', '_max'